In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


from io import StringIO
from html.parser import HTMLParser     
        
class StackOverflowParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.strict = False # If any invalid html is encountered, parser will make a best guess at its intention
        self.convert_charrefs= True # Hold data section until next tag is encountered
        
        # Field variable to keep track of parsed data with tags removed
        self.text = StringIO()
        self.text_no_code = StringIO()
        
        # Field variables to keep track of and store <code></code> blocks
        self.code_blocks = []
        self.lasttag = None
        
    def handle_starttag(self, tag, attrs):
        '''
        Method inherited from HTMLParser super class that is called whenever the start of a tag is encountered.
        In this parser, it keeps track of the last start tag that was encountered.
        :param tag: Current tag being parsed (ex: p, div, code, etc.)
        :type tag: str
        :param attrs: List of (name,value) pairs containing attributes found inside the tag's brackets
        :type attrs: list[str]
        '''
        assert isinstance(tag,str)
        assert isinstance(attrs, list) 
        
        self.lasttag = tag
        
    def handle_data(self, data): 
        '''
        Method inherited from HTMLParser super class that is called whenever data inside of a tag is encountered.
        In this parser, it saves blocks of code to the field variable self.code and records all text with HTML tags removed
        :param data: Current data inside of a tag being parsed
        :type tag: str
        '''
        assert isinstance(data,str)
        
        # If the last tag encountered was a <code> tag, append the contents to the list of code blocks
        if self.lasttag == "code":
            self.lasttag = None
            self.code_blocks.append(data)
        else:  
            self.text_no_code.write(data)
            
        # Record text between tags
        self.text.write(data)
        
    def get_data(self):
        '''
        Returns parsed text without HTML tags 
        :return: Text wihtout tags
        :type return: str
        '''
        return self.text.getvalue()
    
    def get_data_no_code(self):
        '''
        Returns parsed text without HTML tags and with code blocks removed
        :return: Text wihtout tags
        :type return: str
        '''
        return self.text_no_code.getvalue()
    
def strip_tags(html):
    '''
    Takes in a body of text that is formatted in HTML and returns the same text with the HTML tags now removed. 
    This method bundles the process of instantiating a parser, feeding the data, and returning the parsed output.
    :param html: HTML-formatted body of text
    :type html: str
    :return: The input text now without HTML tags
    :type return: str
    '''
    assert isinstance(html,str)
    
    # Feed text into parser and return parsed text without tags
    s = StackOverflowParser()
    s.feed(html)
    return s.get_data()

def get_text_no_code(html):
    '''
    Takes in a body of text that is formatted in HTML and returns the same text with the HTML tags and blocks of code now removed. 
    This method bundles the process of instantiating a parser, feeding the data, and returning the parsed output.
    :param html: HTML-formatted body of text
    :type html: str
    :return: The input text now without HTML tags or code blocks
    :type return: str
    '''
    assert isinstance(html,str)
    
    # Feed text into parser and return parsed text without tags
    s = StackOverflowParser()
    s.feed(html)
    return s.get_data_no_code()

def get_code(html):
    '''
    Takes in a body of text that is formatted in HTML and returns the blocks of code found within the text. 
    This method bundles the process of instantiating a parser, feeding the data, and returning the blocks of code.
    An empty list is returned if no <code> tags are found.
    :param html: HTML-formatted body of text
    :type html: str
    :return: List of blocks of code found within text
    :type return: list[str]
    '''
    assert isinstance(html,str)
    
    s = StackOverflowParser()
    s.feed(html) 
    return [item.replace('\n', ' ') for item in s.code_blocks]
            


# In[ ]:


import pandas as pd

# File paths
file_questions = 'Questions.csv'
file_answers = 'Answers.csv'

# Load dataframes (only loading first 10000 rows for now to reduce processing time)
questions_df = pd.read_csv(file_questions, nrows=100000, encoding = 'iso-8859-1')
answers_df = pd.read_csv(file_answers, nrows=100000, encoding = 'iso-8859-1')


# In[ ]:


# Add extra columns to dataframes
# This takes a long time (~10 minutes) to process the entire dataset
# Might be worth exploring the pandas to_pickle() method for saving/loading the dataframes
questions_df['Body_no_tags']=questions_df['Body'].apply(strip_tags)
questions_df['Body_no_tags_no_code']=questions_df['Body'].apply(get_text_no_code)
questions_df['Body_code']=questions_df['Body'].apply(get_code)

answers_df['Body_no_tags']=answers_df['Body'].apply(strip_tags)
answers_df['Body_no_tags_no_code']=answers_df['Body'].apply(get_text_no_code)
answers_df['Body_code']=answers_df['Body'].apply(get_code)


# In[ ]:


# Row 11 in Questions.csv is a good example with a few code blocks
body = questions_df['Body'][11]
body_no_tags = questions_df['Body_no_tags'][11]
body_code = questions_df['Body_code'][11]
body_no_tags_no_code = questions_df['Body_no_tags_no_code'][11]


# In[ ]:


print(body)


# In[ ]:


print(body_no_tags)


# In[ ]:


print(body_no_tags_no_code)


# In[ ]:


print(body_code)


# In[ ]:


<p>I've got a menu in Python. That part was easy. I'm using <code>raw_input()</code> to get the selection from the user. </p>

<p>The problem is that <code>raw_input</code> (and input) require the user to press <kbd>Enter</kbd> after they make a selection. Is there any way to make the program act immediately upon a keystroke? Here's what I've got so far:</p>

<pre><code>import sys
print """Menu
1) Say Foo
2) Say Bar"""
answer = raw_input("Make a selection&gt; ")

if "1" in answer: print "foo"
elif "2" in answer: print "bar"
</code></pre>

<p>It would be great to have something like</p>

<pre><code>print menu
while lastKey = "":
    lastKey = check_for_recent_keystrokes()
if "1" in lastKey: #do stuff...
</code></pre>

I've got a menu in Python. That part was easy. I'm using raw_input() to get the selection from the user. 

The problem is that raw_input (and input) require the user to press Enter after they make a selection. Is there any way to make the program act immediately upon a key

In [2]:
titles = questions_df['Title']

In [22]:
import itertools

def getWordsAfter(phrase,string):
    '''
    This function parses the input and gets the next word of every
    occurrence of the input "phrase"
    Words after "phrase" seperated by commas are also included
    '''
    words = []
    pos = 0
    while(string.find(phrase,pos) != -1):      
        index = string.find(phrase, pos)                                 #Gets position of phrase
        pos = pos + index + len(phrase) +1                               #Gets next position
        length = 20 if len(string[pos::]) >= 20 else len(string[pos::])  #Ensures no overflow
        word = string[pos:pos+length]
        tokenized = word.split(",")                                      #Split by comma if it has comma
        for word in tokenized:
            ind = 0
            word = word.lstrip()                                         #Strip whitespace
            for char in word:
                if not char.isalpha():                                   #Must only contain letters
                    break
                ind+=1
            word = word[:ind]
            if word:                       
                words.append(word.lower())                               #Append if valid word
        index+=1
    words = list(dict.fromkeys(words))                                   #Removes duplicates (+ Efficiency)
    return words

def getMostCommonLibraries(body_no_tags):
    '''
    This function will search for the most common libraries by 
    looking at keywords after "import" and "from"
    '''
                                                                         #Removes common words
    wordsToRemove = ["the", "that", "a", "d", "it", "i", "python", "e", "an", 
                     "and", "in", "foo", "t", "r", "to", "this", "some", "s",
                     "s", "n", "but", "within", "self", "m", "my", "b", "o",
                     "er", "c", "if", "y", "import", "on"]
    index = 0
    libraries = []
    for i in range(len(body_no_tags)):
        body = body_no_tags[i]
        import_words = getWordsAfter('import',body)                       #Collects words after import
        if import_words:
            libraries.append(import_words)
            from_words = getWordsAfter('from',body)  #if there is a "from", there is also an "import"
            if from_words:
                libraries.append(from_words)
        
    lib_flat = list(itertools.chain.from_iterable(libraries))             #Flattens list
    lib_flat =[word for word in lib_flat if word not in wordsToRemove]    #Removes words from wordsToRemove
    return lib_flat


body_no_tags = questions_df['Body_no_tags']

#print(getMostCommonLibraries(body_no_tags))

In [23]:
from collections import Counter

def getMostCommonElementsInList(aList, numElements = 30):
    c = Counter(aList)
    return c.most_common(numElements)
    
libraries = getMostCommonLibraries(body_no_tags)
common = getMostCommonElementsInList(libraries)
#print(common)

[('sys', 1638), ('os', 1443), ('django', 981), ('nt', 951), ('urllib', 722), ('numpy', 700), ('re', 627), ('time', 548), ('ng', 523), ('subprocess', 396), ('random', 379), ('datetime', 350), ('matplotlib', 332), ('socket', 327), ('models', 306), ('wx', 298), ('csv', 280), ('beautifulsoup', 269), ('tkinter', 258), ('google', 241), ('xml', 225), ('pyqt', 220), ('threading', 216), ('line', 213), ('math', 207), ('string', 205), ('win', 202), ('setup', 192), ('file', 178), ('lxml', 177)]


In [5]:
def getMostCommonCategory(titles, body_no_tags, categories):
    categoryCount = dict()
    for key in categories.keys():
        categoryCount[key] = 0
    for i in range(len(body_no_tags)):
        title = titles[i].lower()
        body = body_no_tags[i].lower()
        for key in categories.keys():
            for phrase in categories[key]:
                if title.find(phrase) != -1 or body.find(phrase) != -1:
                    categoryCount[key]+=1
                    break
    return categoryCount

In [6]:
def getMostCommonOS(titles, body_no_tags):
    os = dict()
    os["windows"] = ["windows", "microsoft"]
    os["mac"] = ["mac", "macos", "macintosh"]
    os["linux"] = ["linux", "unix", "ubuntu", "debian", "gentoo", "rhel", "centos", "fedora", "kali", "arch", "kubuntu", "deepin"]
    os["chrome"] = ["chromium", "chrome OS", "chromeos"]
    categoryCount = getMostCommonCategory(titles,body_no_tags, os)
    return categoryCount
def getMostCommonIDE(titles, body_no_tags):
    ide = dict()
    ide["jupyter"] = ["jupyter"]
    ide["pycharm"] = ["pycharm"]
    ide["spyder"] = ["spyder"]
    ide["vscode"] = ["visual studio", "visualstudio", "vscode", "vs code"] #Also includes visual studio
    ide["sublime"] = ["sublime"]
    ide["atom"] = ["atom"]
    ide["vim"] = ["vim"]
    ide["eclipse"] = ["eclipse"]
    ide["emacs"] = ["emacs"]
    ide["gedit"] = ["gedit"]
    ide["rodeo"] = ["rodeo"]
    ide["notepad++"] = ["notepad++"]
    ide["intellij"] = ["intellij"]
    ide["xcode"] = ["xcode"]
    ide["phpstorm"] = ["phpstorm"]
    ide["netbeans"] = ["netbeans"]
    categoryCount = getMostCommonCategory(titles, body_no_tags, ide)
    return categoryCount
def getMostCommonPM(titles, body_no_tags): #PM = Package Manager 
    pm = dict() 
    pm["pip"] = ["pip"]
    pm["conda"] = ["conda"] 
    categoryCount = getMostCommonCategory(titles,body_no_tags, pm)
    return categoryCount

os = getMostCommonOS(titles, body_no_tags)
for key in os.keys():
    print(key + ":" , os[key])
print("\n")
ide = getMostCommonIDE(titles, body_no_tags)
for key in ide.keys():
    print(key + ":" , ide[key])
print("\n")
pm = getMostCommonPM(titles, body_no_tags)
for key in pm.keys():
    print(key + ":" , pm[key])

windows: 6258
mac: 5402
linux: 13576
chrome: 33


jupyter: 1
pycharm: 116
spyder: 32
vscode: 277
sublime: 22
atom: 291
vim: 358
eclipse: 776
emacs: 253
gedit: 100
rodeo: 3
notepad++: 62
intellij: 13
xcode: 167
phpstorm: 1
netbeans: 103


pip: 2368
conda: 207


In [7]:
from collections import OrderedDict 

titlesByYear = OrderedDict()
questionsByYear = OrderedDict()
questionDate = questions_df['CreationDate']
index = 0
for q in questionDate:
    year = q[:7]
    if year not in titlesByYear.keys():
        titlesByYear[year] = []
    if year not in questionsByYear.keys():
        questionsByYear[year] = []
    titlesByYear[year].append(titles[index])
    questionsByYear[year].append(body_no_tags[index])
    index+=1

print (list(questionsByYear.items())[0])

('2008-08', ["I am using the Photoshop's javascript API to find the fonts in a given PSD.\n\nGiven a font name returned by the API, I want to find the actual physical font file that that font name corresponds to on the disc.\n\nThis is all happening in a python program running on OSX so I guess I'm looking for one of:\n\n\nSome Photoshop javascript\nA Python function\nAn OSX API that I can call from python\n\n", 'I have a cross-platform (Python) application which needs to generate a JPEG preview of the first page of a PDF.\n\nOn the Mac I am spawning sips.  Is there something similarly simple I can do on Windows?\n', "I'm starting work on a hobby project with a python codebase and would like to set up some form of continuous integration (i.e. running a battery of test-cases each time a check-in is made and sending nag e-mails to responsible persons when the tests fail) similar to CruiseControl or TeamCity.\n\nI realize I could do this with hooks in most VCSes, but that requires that th

In [12]:
commonLibrariesByYear = dict()
commonOSByYear = dict()
commonIDEByYear = dict()
commonPMByYear = dict()

for year in questionsByYear.keys():
    libraries = getMostCommonLibraries(questionsByYear[year])
    common = getMostCommonElementsInList(libraries,15)
    commonLibrariesByYear[year] = common
    commonOSByYear[year] = getMostCommonOS(titlesByYear[year],questionsByYear[year])
    commonIDEByYear[year] = getMostCommonIDE(titlesByYear[year], questionsByYear[year])
    commonPMByYear[year] = getMostCommonPM(titlesByYear[year], questionsByYear[year])

OSPieInput = dict()
PMPieInput = dict()

libraryBarInput = OrderedDict()
libraryLabelsList = []
libraryValuesList = []
for year in questionsByYear.keys():
    
    OSPieInputList = []
    PMPieInputList = []
    tempLabels = []
    tempValues = []
    print("Year: ",year)
    print(commonLibrariesByYear[year])
    for pair in commonLibrariesByYear[year]:
        tempLabels.append(pair[0])
        tempValues.append(pair[1])
    libraryLabelsList.append(tempLabels)
    libraryBarInput[year] = tempValues

    for key in commonOSByYear[year].keys():
        print(key + ":" , commonOSByYear[year][key])
        OSPieInputList.append(commonOSByYear[year][key])
    print("\n")
    for key in commonIDEByYear[year].keys():
        print(key + ":" , commonIDEByYear[year][key])
    print("\n")
    for key in commonPMByYear[year].keys():
        print(key + ":" , commonPMByYear[year][key])
        PMPieInputList.append(commonPMByYear[year][key])
    print("\n")
    OSPieInput[year] = OSPieInputList
    PMPieInput[year] = PMPieInputList
#print(OSPieInput)
#print(len(OSPieInput))

print(libraryLabelsList)

Year:  2008-08
[('re', 3), ('os', 3), ('sys', 2), ('ftp', 1), ('ftpuser', 1), ('serial', 1), ('urllib', 1), ('html', 1), ('rl', 1), ('zipfile', 1), ('time', 1), ('unittest', 1), ('subprocess', 1), ('k', 1), ('nters', 1)]
windows: 11
mac: 7
linux: 14
chrome: 0


jupyter: 0
pycharm: 0
spyder: 0
vscode: 1
sublime: 0
atom: 0
vim: 0
eclipse: 0
emacs: 0
gedit: 0
rodeo: 0
notepad++: 0
intellij: 0
xcode: 0
phpstorm: 0
netbeans: 0


pip: 2
conda: 0


Year:  2008-09
[('re', 4), ('nt', 3), ('django', 3), ('datetime', 3), ('ng', 2), ('decimal', 2), ('cgi', 1), ('dbi', 1), ('odbc', 1), ('its', 1), ('app', 1), ('xml', 1), ('smtp', 1), ('om', 1), ('causi', 1)]
windows: 41
mac: 33
linux: 58
chrome: 0


jupyter: 0
pycharm: 0
spyder: 0
vscode: 3
sublime: 0
atom: 0
vim: 4
eclipse: 2
emacs: 1
gedit: 0
rodeo: 0
notepad++: 1
intellij: 0
xcode: 2
phpstorm: 0
netbeans: 0


pip: 3
conda: 1


Year:  2008-10
[('nt', 5), ('sys', 5), ('re', 3), ('os', 3), ('ng', 2), ('commands', 2), ('models', 2), ('django', 2), (

mac: 124
linux: 308
chrome: 0


jupyter: 0
pycharm: 1
spyder: 0
vscode: 5
sublime: 1
atom: 2
vim: 7
eclipse: 25
emacs: 4
gedit: 0
rodeo: 0
notepad++: 0
intellij: 0
xcode: 2
phpstorm: 0
netbeans: 3


pip: 47
conda: 11


Year:  2010-09
[('nt', 31), ('sys', 31), ('django', 26), ('os', 20), ('urllib', 18), ('re', 14), ('time', 14), ('socket', 11), ('ng', 11), ('numpy', 9), ('datetime', 8), ('models', 8), ('module', 7), ('user', 6), ('wx', 6)]
windows: 151
mac: 111
linux: 284
chrome: 0


jupyter: 0
pycharm: 1
spyder: 0
vscode: 7
sublime: 0
atom: 5
vim: 11
eclipse: 8
emacs: 9
gedit: 0
rodeo: 0
notepad++: 1
intellij: 0
xcode: 2
phpstorm: 0
netbeans: 2


pip: 44
conda: 7


Year:  2010-10
[('sys', 37), ('os', 37), ('nt', 28), ('ng', 16), ('urllib', 15), ('datetime', 14), ('socket', 13), ('random', 13), ('django', 12), ('time', 12), ('threading', 10), ('beautifulsoup', 10), ('tkinter', 10), ('models', 10), ('re', 9)]
windows: 165
mac: 103
linux: 317
chrome: 2


jupyter: 0
pycharm: 3
spyder: 0
vs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


from collections import OrderedDict 
from easing import easing


def createAnimatedPieChart(labels, data,startYear):
    explode=[.01]*len(labels)
    nums =[0]*len(labels)
    fig, ax = plt.subplots()

    def update(num):
        ax.clear()
        ax.axis('equal')
        nums = list(data.items())[num][1]
        print(nums)
        ax.pie(nums, explode=explode, labels=labels, autopct='%1.1f%%', shadow=False, startangle=140)
        ax.set_title('Year: ' + str((list(data.keys())[num])))
        time.sleep(.25)
    ani = FuncAnimation(fig, update, frames=range(len(data)), repeat=False, interval = 200)
    plt.show()

#OSPieInput = {2008: [165, 120, 252, 0], 2009: [908, 685, 1554, 1], 2010: [1649, 1371, 3363, 8], 2011: [2473, 2227, 5793, 19], 2012: [1063, 999, 2614, 5]}
OSPieInput = OrderedDict({'2008-08': [11, 7, 14, 0], '2008-09': [41, 33, 58, 0], '2008-10': [48, 38, 73, 0], '2008-11': [38, 21, 63, 0], '2008-12': [27, 21, 44, 0], '2009-01': [48, 34, 76, 0], '2009-02': [54, 34, 70, 0], '2009-03': [45, 39, 91, 0], '2009-04': [57, 32, 95, 0], '2009-05': [76, 59, 112, 0], '2009-06': [75, 56, 124, 0], '2009-07': [97, 54, 144, 0], '2009-08': [82, 65, 139, 0], '2009-09': [81, 74, 142, 1], '2009-10': [91, 71, 171, 0], '2009-11': [93, 76, 186, 0], '2009-12': [109, 91, 204, 0], '2010-01': [116, 89, 234, 0], '2010-02': [112, 96, 226, 0], '2010-03': [120, 88, 265, 1], '2010-04': [103, 107, 212, 0], '2010-05': [145, 92, 231, 1], '2010-06': [127, 137, 280, 0], '2010-07': [168, 134, 335, 0], '2010-08': [149, 124, 308, 0], '2010-09': [151, 111, 284, 0], '2010-10': [165, 103, 317, 2], '2010-11': [151, 157, 339, 1], '2010-12': [142, 133, 332, 3], '2011-01': [185, 152, 404, 1], '2011-02': [188, 167, 430, 0], '2011-03': [225, 183, 476, 0], '2011-04': [154, 166, 460, 0], '2011-05': [208, 193, 515, 2], '2011-06': [200, 171, 480, 0], '2011-07': [219, 213, 520, 5], '2011-08': [249, 218, 518, 2], '2011-09': [211, 178, 460, 0], '2011-10': [212, 180, 500, 3], '2011-11': [235, 206, 547, 3], '2011-12': [187, 200, 483, 3], '2012-01': [219, 209, 585, 2], '2012-02': [279, 271, 679, 1], '2012-03': [258, 251, 674, 1], '2012-04': [282, 250, 641, 1], '2012-05': [25, 18, 35, 0]})
labels = ['Windows', 'Mac', 'Linux', 'ChromeOS']
#createAnimatedPieChart(labels,OSPieInput,2008)

#data = np.array(list(OSPieInput.items())[0][1])
#print(data)
#out_data = easing.Eased(data).power_ease(3).flatten()
#print(out_data)
#data=np.random.random((10,2))
#easing.Eased(data).scatter_animation2d(n=3,speed=0.5,destination='singlepoint.gif')

import matplotlib.pyplot as plt
from matplotlib import animation

import pandas as pd

def createAnimatedBarPlot(labels, data, sleepTime = 0):
    fig, ax = plt.subplots()
    #colors = plt.cm.Dark2(range(6))
    maxValue = 0
    for key in data.keys():
        print(max(data[key]))
        if maxValue < max(data[key]):
            maxValue = max(data[key])
    print(maxValue)
    def animate(i):
        x = labels
        if(isinstance(labels[0],list)):
            x = labels[i]
        ax.clear()
        ax.set_title(str((list(data.keys())[i])))        
        width = list(data.items())[i][1]
        df = pd.DataFrame({"Labels":x, "Data":width})
        df_sorted= df.sort_values('Data')
        #plt.barh(y=labels, width = width, align = 'center', color = colors)
        plt.barh('Labels', 'Data', data=df_sorted)
        plt.xlim(0, 1.1*maxValue)
        time.sleep(sleepTime)
    anim=animation.FuncAnimation(fig,animate, repeat = False, frames=len(data))
    plt.show()

OSPieInput = OrderedDict({'2008-08': [11, 7, 14, 0], '2008-09': [41, 33, 58, 0], '2008-10': [48, 38, 73, 0], '2008-11': [38, 21, 63, 0], '2008-12': [27, 21, 44, 0], '2009-01': [48, 34, 76, 0], '2009-02': [54, 34, 70, 0], '2009-03': [45, 39, 91, 0], '2009-04': [57, 32, 95, 0], '2009-05': [76, 59, 112, 0], '2009-06': [75, 56, 124, 0], '2009-07': [97, 54, 144, 0], '2009-08': [82, 65, 139, 0], '2009-09': [81, 74, 142, 1], '2009-10': [91, 71, 171, 0], '2009-11': [93, 76, 186, 0], '2009-12': [109, 91, 204, 0], '2010-01': [116, 89, 234, 0], '2010-02': [112, 96, 226, 0], '2010-03': [120, 88, 265, 1], '2010-04': [103, 107, 212, 0], '2010-05': [145, 92, 231, 1], '2010-06': [127, 137, 280, 0], '2010-07': [168, 134, 335, 0], '2010-08': [149, 124, 308, 0], '2010-09': [151, 111, 284, 0], '2010-10': [165, 103, 317, 2], '2010-11': [151, 157, 339, 1], '2010-12': [142, 133, 332, 3], '2011-01': [185, 152, 404, 1], '2011-02': [188, 167, 430, 0], '2011-03': [225, 183, 476, 0], '2011-04': [154, 166, 460, 0], '2011-05': [208, 193, 515, 2], '2011-06': [200, 171, 480, 0], '2011-07': [219, 213, 520, 5], '2011-08': [249, 218, 518, 2], '2011-09': [211, 178, 460, 0], '2011-10': [212, 180, 500, 3], '2011-11': [235, 206, 547, 3], '2011-12': [187, 200, 483, 3], '2012-01': [219, 209, 585, 2], '2012-02': [279, 271, 679, 1], '2012-03': [258, 251, 674, 1], '2012-04': [282, 250, 641, 1], '2012-05': [25, 18, 35, 0]})
labels = ['Windows', 'Mac', 'Linux', 'ChromeOS']
#createAnimatedBarPlot(labels,OSPieInput)

PMPieInput = OrderedDict({'2008-08': [2, 0], '2008-09': [3, 1], '2008-10': [5, 1], '2008-11': [2, 2], '2008-12': [10, 0], '2009-01': [8, 2], '2009-02': [6, 0], '2009-03': [15, 3], '2009-04': [13, 0], '2009-05': [16, 3], '2009-06': [17, 0], '2009-07': [21, 2], '2009-08': [16, 1], '2009-09': [23, 0], '2009-10': [25, 1], '2009-11': [27, 0], '2009-12': [27, 6], '2010-01': [35, 5], '2010-02': [28, 6], '2010-03': [34, 5], '2010-04': [26, 2], '2010-05': [29, 3], '2010-06': [40, 2], '2010-07': [34, 3], '2010-08': [47, 11], '2010-09': [44, 7], '2010-10': [51, 3], '2010-11': [58, 4], '2010-12': [50, 3], '2011-01': [63, 7], '2011-02': [82, 4], '2011-03': [91, 3], '2011-04': [80, 2], '2011-05': [96, 8], '2011-06': [88, 8], '2011-07': [86, 11], '2011-08': [106, 6], '2011-09': [95, 7], '2011-10': [96, 6], '2011-11': [97, 8], '2011-12': [103, 9], '2012-01': [132, 9], '2012-02': [126, 16], '2012-03': [167, 13], '2012-04': [135, 14], '2012-05': [13, 0]})
labels = ["Pip","Conda"]
#createAnimatedBarPlot(labels,PMPieInput)

libraryBarInput =OrderedDict([('2008-08', [3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), ('2008-09', [4, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]), ('2008-10', [5, 5, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), ('2008-11', [6, 5, 5, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]), ('2008-12', [6, 5, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2]), ('2009-01', [11, 10, 6, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2]), ('2009-02', [9, 7, 6, 5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2]), ('2009-03', [12, 12, 10, 9, 7, 5, 5, 4, 4, 4, 3, 2, 2, 2, 2]), ('2009-04', [11, 10, 10, 7, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3]), ('2009-05', [15, 15, 11, 6, 6, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3]), ('2009-06', [17, 11, 10, 9, 7, 6, 5, 5, 5, 5, 5, 5, 4, 3, 3]), ('2009-07', [19, 19, 12, 10, 7, 6, 5, 5, 5, 5, 4, 4, 4, 3, 3]), ('2009-08', [12, 12, 9, 8, 8, 7, 6, 6, 5, 5, 4, 4, 4, 4, 4]), ('2009-09', [12, 11, 11, 10, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5]), ('2009-10', [22, 17, 12, 11, 11, 11, 9, 8, 7, 6, 5, 5, 5, 5, 5]), ('2009-11', [26, 24, 13, 11, 11, 9, 9, 7, 7, 6, 6, 6, 6, 5, 5]), ('2009-12', [24, 23, 16, 13, 12, 10, 8, 8, 8, 8, 6, 6, 6, 5, 5]), ('2010-01', [31, 25, 20, 20, 12, 11, 10, 10, 9, 8, 8, 7, 7, 6, 6]), ('2010-02', [21, 21, 20, 20, 11, 11, 10, 9, 9, 9, 7, 6, 6, 6, 5]), ('2010-03', [27, 24, 17, 16, 14, 12, 12, 11, 11, 10, 10, 9, 7, 6, 6]), ('2010-04', [25, 19, 15, 14, 14, 12, 10, 9, 7, 7, 6, 6, 6, 5, 5]), ('2010-05', [33, 32, 25, 19, 15, 13, 11, 9, 9, 7, 7, 6, 6, 6, 6]), ('2010-06', [39, 21, 19, 19, 16, 15, 11, 9, 9, 8, 8, 8, 6, 6, 6]), ('2010-07', [23, 23, 21, 18, 17, 14, 14, 13, 13, 12, 10, 10, 9, 9, 8]), ('2010-08', [43, 33, 22, 21, 16, 15, 15, 13, 9, 8, 8, 8, 7, 7, 7]), ('2010-09', [31, 31, 26, 20, 18, 14, 14, 11, 11, 9, 8, 8, 7, 6, 6]), ('2010-10', [37, 37, 28, 16, 15, 14, 13, 13, 12, 12, 10, 10, 10, 10, 9]), ('2010-11', [41, 34, 24, 22, 14, 14, 13, 12, 12, 12, 11, 10, 9, 9, 7]), ('2010-12', [35, 28, 25, 24, 16, 15, 12, 11, 10, 10, 10, 9, 9, 8, 8]), ('2011-01', [41, 38, 36, 36, 21, 19, 17, 17, 16, 14, 12, 12, 9, 9, 9]), ('2011-02', [59, 56, 42, 23, 21, 20, 17, 16, 16, 16, 14, 14, 14, 13, 12]), ('2011-03', [63, 44, 35, 31, 31, 29, 20, 19, 16, 14, 14, 13, 13, 13, 13]), ('2011-04', [62, 52, 34, 32, 29, 24, 18, 16, 15, 15, 15, 14, 13, 13, 13]), ('2011-05', [68, 57, 39, 37, 25, 23, 20, 19, 18, 17, 17, 15, 13, 12, 11]), ('2011-06', [52, 37, 36, 35, 32, 21, 20, 19, 19, 18, 15, 15, 15, 13, 12]), ('2011-07', [69, 59, 36, 24, 24, 21, 20, 20, 19, 17, 17, 16, 13, 11, 10]), ('2011-08', [67, 58, 51, 39, 32, 32, 31, 25, 20, 20, 18, 18, 17, 15, 14]), ('2011-09', [72, 65, 33, 30, 30, 28, 27, 19, 18, 17, 16, 16, 15, 13, 13]), ('2011-10', [58, 53, 44, 34, 26, 25, 23, 22, 22, 21, 20, 20, 14, 14, 13]), ('2011-11', [77, 50, 39, 39, 35, 27, 20, 20, 18, 17, 16, 16, 16, 14, 14]), ('2011-12', [68, 50, 32, 32, 31, 29, 25, 24, 24, 23, 20, 19, 17, 16, 15]), ('2012-01', [78, 57, 49, 43, 39, 32, 27, 26, 25, 23, 23, 18, 17, 17, 15]), ('2012-02', [95, 77, 67, 42, 40, 39, 36, 36, 26, 23, 23, 20, 19, 18, 17]), ('2012-03', [93, 82, 52, 48, 37, 36, 34, 31, 30, 29, 23, 22, 20, 19, 18]), ('2012-04', [87, 72, 56, 52, 49, 44, 35, 29, 27, 23, 22, 21, 21, 20, 19]), ('2012-05', [8, 6, 6, 5, 5, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2])])
libraryLabels = [['re', 'os', 'sys', 'ftp', 'ftpuser', 'serial', 'urllib', 'html', 'rl', 'zipfile', 'time', 'unittest', 'subprocess', 'k', 'nters'], ['re', 'nt', 'django', 'datetime', 'ng', 'decimal', 'cgi', 'dbi', 'odbc', 'its', 'app', 'xml', 'smtp', 'om', 'causi'], ['nt', 'sys', 're', 'os', 'ng', 'commands', 'models', 'django', 'cherrypy', 'feedparser', 'nd', 'modules', 'getopt', 'ar', 'itertools'], ['sys', 'wx', 're', 'os', 'ng', 'nt', 'django', 'lf', 'so', 'ctypes', 'mechanize', 'models', 'zipfile', 'configparser', 'xml'], ['nt', 'django', 'mysqldb', 're', 'os', 'sys', 'random', 'numpy', 'site', 'ng', 'ort', 'string', 'eggs', 'another', 'urllib'], ['sys', 'os', 'nt', 'urllib', 'time', 'thread', 're', 'pprint', 'file', 'all', 'django', 'xml', 'ng', 'code', 'xmlrpclib'], ['sys', 'math', 'os', 'django', 'nt', 're', 'time', 'ng', 'wx', 'urllib', 'datetime', 'facebook', 'xml', 'them', 'unittest'], ['nt', 'os', 'sys', 'django', 'urllib', 'ng', 'win', 're', 'time', 'wx', 'datetime', 'weakref', 'beautifulsoup', 'user', 'he'], ['os', 'nt', 'sys', 'time', 'socket', 'pygame', 'module', 'django', 'urllib', 'datetime', 'popen', 'pipe', 'stdout', 'httplib', 'ng'], ['django', 'sys', 'nt', 'urllib', 're', 'ng', 'beautifulsoup', 'file', 'image', 'os', 'threading', 'le', 'setup', 'win', 'models'], ['os', 'sys', 're', 'django', 'nt', 'time', 'ng', 'numpy', 'csv', 'wx', 'math', 'urllib', 'subprocess', 'sqlite', 'http'], ['os', 'sys', 'nt', 'django', 'urllib', 'time', 'ng', 'wx', 'subprocess', 'models', 'of', 'ntly', 'xml', 'thread', 'win'], ['sys', 'nt', 'django', 'os', 'urllib', 're', 'ng', 'datetime', 'string', 'timeit', 'for', 'client', 'unittest', 'socket', 'pyqt'], ['os', 'django', 'sys', 'nt', 'ng', 'time', 'google', 'socket', 'math', 'datetime', 'models', 'subprocess', 're', 'urllib', 'sqlite'], ['os', 'sys', 'numpy', 're', 'django', 'ng', 'nt', 'urllib', 'threading', 'time', 'random', 'xml', 'subprocess', 'selenium', 'datetime'], ['os', 'sys', 'nt', 'django', 're', 'urllib', 'ng', 'time', 'math', 'is', 'google', 'datetime', 'httplib', 'unittest', 'numpy'], ['os', 'sys', 'django', 'urllib', 'nt', 're', 'time', 'subprocess', 'pyqt', 'google', 'random', 'urlfetch', 'image', 'threading', 'socket'], ['os', 'sys', 'nt', 'django', 'urllib', 're', 'time', 'ng', 'models', 'datetime', 'matplotlib', 'image', 'threading', 'wx', 'scipy'], ['nt', 'sys', 'django', 'os', 'models', 'time', 'ng', 'wx', 'sqlalchemy', 'urllib', 'numpy', 'f', 're', 'l', 'module'], ['sys', 'os', 'django', 'nt', 'numpy', 're', 'time', 'subprocess', 'wx', 'beautifulsoup', 'matplotlib', 'urllib', 'google', 'or', 'string'], ['sys', 'os', 'nt', 'urllib', 'numpy', 'django', 'google', 're', 'ng', 'socket', 'time', 'line', 'win', 'wx', 'hashlib'], ['sys', 'django', 'os', 'nt', 'datetime', 'urllib', 'socket', 'ng', 're', 'time', 'string', 'template', 'lxml', 'xml', 'image'], ['sys', 'nt', 'os', 'urllib', 'django', 're', 'ng', 'google', 'wx', 'numpy', 'math', 'time', 'lxml', 'models', 'db'], ['nt', 'os', 'sys', 'django', 'urllib', 'time', 'csv', 'numpy', 'ng', 'socket', 'matplotlib', 'wx', 'beautifulsoup', 'subprocess', 'threading'], ['sys', 'os', 'django', 'nt', 'time', 'ng', 'urllib', 'numpy', 'wx', 'twisted', 'selenium', 'nltk', 'setup', 'module', 'gtk'], ['nt', 'sys', 'django', 'os', 'urllib', 're', 'time', 'socket', 'ng', 'numpy', 'datetime', 'models', 'module', 'user', 'wx'], ['sys', 'os', 'nt', 'ng', 'urllib', 'datetime', 'socket', 'random', 'django', 'time', 'threading', 'beautifulsoup', 'tkinter', 'models', 're'], ['sys', 'os', 'django', 'nt', 'urllib', 'subprocess', 're', 'numpy', 'ng', 'tkinter', 'google', 'models', 'time', 'lxml', 'beautifulsoup'], ['sys', 'django', 'nt', 'os', 'numpy', 'urllib', 'ng', 'line', 'random', 're', 'models', 'subprocess', 'time', 'multiprocessing', 'distutils'], ['django', 'nt', 'sys', 'os', 'numpy', 'ng', 'urllib', 'time', 're', 'models', 'random', 'socket', 'mysqldb', 'beautifulsoup', 'lxml'], ['os', 'sys', 'django', 'urllib', 'numpy', 'nt', 'matplotlib', 'random', 're', 'socket', 'models', 'subprocess', 'time', 'ng', 'beautifulsoup'], ['sys', 'os', 'nt', 'django', 'urllib', 'numpy', 're', 'time', 'datetime', 'matplotlib', 'models', 'subprocess', 'line', 'setup', 'random'], ['sys', 'os', 'numpy', 'urllib', 'django', 'nt', 'ng', 'string', 'random', 're', 'subprocess', 'socket', 'time', 'google', 'beautifulsoup'], ['sys', 'os', 'nt', 'django', 're', 'time', 'matplotlib', 'subprocess', 'ng', 'numpy', 'wx', 'datetime', 'random', 'urllib', 'mysqldb'], ['sys', 'os', 'urllib', 'django', 're', 'ng', 'nt', 'datetime', 'matplotlib', 'socket', 'subprocess', 'time', 'numpy', 'def', 'pyqt'], ['os', 'sys', 'nt', 'django', 'time', 'matplotlib', 'datetime', 'urllib', 'numpy', 'ng', 're', 'random', 'subprocess', 'csv', 'tkinter'], ['sys', 'os', 'django', 'nt', 'urllib', 'numpy', 're', 'beautifulsoup', 'ng', 'subprocess', 'time', 'socket', 'models', 'random', 'csv'], ['os', 'sys', 'numpy', 're', 'nt', 'django', 'urllib', 'datetime', 'subprocess', 'tkinter', 'time', 'ng', 'random', 'pygame', 'xml'], ['os', 'sys', 'nt', 'numpy', 'urllib', 'ng', 'time', 'django', 'random', 're', 'socket', 'csv', 'beautifulsoup', 'xml', 'subprocess'], ['sys', 'os', 'django', 'nt', 'numpy', 'urllib', 're', 'subprocess', 'wx', 'ng', 'datetime', 'matplotlib', 'random', 'xml', 'time'], ['sys', 'os', 'nt', 'time', 'django', 'urllib', 'numpy', 'csv', 're', 'ng', 'random', 'pygame', 'matplotlib', 'socket', 'tkinter'], ['sys', 'os', 'django', 'numpy', 'nt', 're', 'urllib', 'ng', 'time', 'random', 'matplotlib', 'pygame', 'pyqt', 'wx', 'subprocess'], ['sys', 'os', 'numpy', 'django', 're', 'urllib', 'time', 'nt', 'ng', 'subprocess', 'matplotlib', 'random', 'csv', 'tkinter', 'math'], ['os', 'sys', 'django', 'numpy', 'nt', 're', 'urllib', 'ng', 'time', 'subprocess', 'random', 'datetime', 'csv', 'matplotlib', 'beautifulsoup'], ['sys', 'os', 'numpy', 'django', 'urllib', 'nt', 're', 'time', 'socket', 'random', 'datetime', 'beautifulsoup', 'subprocess', 'matplotlib', 'tkinter'], ['os', 'time', 'subprocess', 'numpy', 'urllib', 'django', 'beautifulsoup', 'cv', 'sqlite', 'sys', 'def', 'on', 'scrapy', 'codecs', 'wx']]
createAnimatedBarPlot(libraryLabels,libraryBarInput,.5)